In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from sklearn.linear_model import LogisticRegression, LinearRegression
%matplotlib inline

## functions

In [2]:
def Metric_Comp(predict, true):
    error = predict - true
    error = np.absolute(error)
    error = error/true
    error = np.mean(error)
    return 1 - error

In [3]:
def train_predict(model, X, Y, X_Vali):
    model_fitted = model.fit(X,Y)
    Y_vali = model_fitted.predict(X_vali)
    return model_fitted, Y_vali

In [4]:
def test_predict(model, X_test):
    Y_test = model.predict(X_test)
    return Y_test

## preliminary analysis

In [5]:
train_data = pd.read_csv('..\\..\\Bitcoint_Data\\train_timeseries.csv')

In [6]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [7]:
train_data.head()

,date,open,high,low,close,market_cap,avg_block_size,blocks_size,cost_per_transaction_percent,cost_per_transaction,...,transactions_per_sec_min,transactions_per_sec_max,utxo_count_mean,utxo_count_stddev,utxo_count_min,utxo_count_max,news_stories_total,news_stories_positive,news_stories_negative,news_stories_neutral
0,2018-11-30,4289.09,4322.98,3942.82,4017.27,6.990464e+10,1.209791,192791.5434,0.328777,21.396147,...,1.183333,17.700000,58319967.38,27106.89859,58281148,58355400,1071,356,549,166
1,2018-11-29,4269.00,4413.02,4145.77,4278.85,7.445102e+10,1.202130,192669.3545,0.625105,23.508462,...,1.283333,27.916667,58252488.42,15831.47820,58212951,58269350,1096,389,521,186
2,2018-11-28,3822.47,4385.90,3822.47,4257.42,7.407256e+10,1.111968,192543.1309,0.762892,24.411103,...,0.433333,33.850000,58186459.38,13193.95528,58165046,58213488,1187,409,583,195
3,2018-11-27,3765.95,3862.96,3661.01,3820.72,6.646897e+10,1.071254,192411.9186,0.730799,23.399753,...,1.683333,17.083333,58159562.88,9943.16102,58132945,58176825,1239,424,597,218
4,2018-11-26,4015.07,4107.14,3643.92,3779.13,6.573929e+10,1.186164,192271.5843,0.741989,21.745022,...,1.166667,27.200000,58120822.67,13528.76274,58082636,58135200,1343,454,670,219


In [24]:
'''splitting the data into training and validation
1. Seperate the recent 120 examples as the validation
'''
seper = 120
starter = train_data.iloc[0,0]
validation_list = [starter - timedelta(days = i) for i in range(seper)]
validation_data = train_data[train_data['date'].isin(validation_list)].copy()
training_data = train_data[~train_data['date'].isin(validation_list)].copy()

training_data.to_csv('..\\..\\intermediate_files\\training_data.csv', index=None)
validation_data.to_csv('..\\..\\intermediate_files\\validation_data.csv', index=None)

## reading the files for model

In [8]:
train_data = pd.read_csv('..\\..\\intermediate_files\\training_data.csv')
validation_data = pd.read_csv('..\\..\\intermediate_files\\validation_data.csv')

In [9]:
train_data.dtypes

date                                           object
open                                          float64
high                                          float64
low                                           float64
close                                         float64
market_cap                                    float64
avg_block_size                                float64
blocks_size                                   float64
cost_per_transaction_percent                  float64
cost_per_transaction                          float64
difficulty                                    float64
est_transaction_volume_USD                    float64
est_transaction_volume                        float64
hash_rate                                     float64
median_confirmation_time                      float64
miners_revenue                                float64
n_transactions_excl_chains_longer_than_100      int64
n_transactions_excl_popular                     int64
n_transactions_per_block    

In [10]:
model_log = LinearRegression()
X = train_data[['open','high','low']].values
Y = train_data['close'].values
X_vali = validation_data[['open','high','low']].values

In [11]:
model_log, Y_vali = train_predict(model_log, X, Y, X_vali)

In [12]:
vali_predict = model_log.predict(validation_data[['open','high','low']].values)

In [13]:
vali_predict

array([ 4089.15070368,  4297.51941121,  4319.59550066,  3765.81951416,
        3858.98333199,  3907.65645273,  4061.64309871,  4262.9040611 ,
        4449.28021464,  4555.19611233,  4571.37607225,  5133.63398835,
        5595.93677102,  5493.21633816,  5521.48489433,  5507.55479105,
        5859.01492642,  6315.98208101,  6342.88977853,  6325.86187424,
        6372.7424021 ,  6355.2920445 ,  6426.74793921,  6485.19586145,
        6386.10254367,  6412.92207938,  6286.8184715 ,  6313.15769776,
        6306.01604691,  6469.91915212,  6276.65014819,  6287.77050121,
        6337.21412478,  6419.59567721,  6428.49450446,  6424.05137792,
        6420.31985116,  6451.78082315,  6430.24631718,  6465.38110099,
        6482.43411072,  6427.01564703,  6412.38942963,  6451.76414851,
        6496.49093923,  6589.19869421,  6846.87226576,  6294.15309237,
        6236.59869743,  6262.23149567,  6331.98393777,  6519.88013698,
        6571.18988623,  6593.81361997,  6556.44791832,  6539.61170035,
      

In [14]:
Y_vali

array([ 4089.15070368,  4297.51941121,  4319.59550066,  3765.81951416,
        3858.98333199,  3907.65645273,  4061.64309871,  4262.9040611 ,
        4449.28021464,  4555.19611233,  4571.37607225,  5133.63398835,
        5595.93677102,  5493.21633816,  5521.48489433,  5507.55479105,
        5859.01492642,  6315.98208101,  6342.88977853,  6325.86187424,
        6372.7424021 ,  6355.2920445 ,  6426.74793921,  6485.19586145,
        6386.10254367,  6412.92207938,  6286.8184715 ,  6313.15769776,
        6306.01604691,  6469.91915212,  6276.65014819,  6287.77050121,
        6337.21412478,  6419.59567721,  6428.49450446,  6424.05137792,
        6420.31985116,  6451.78082315,  6430.24631718,  6465.38110099,
        6482.43411072,  6427.01564703,  6412.38942963,  6451.76414851,
        6496.49093923,  6589.19869421,  6846.87226576,  6294.15309237,
        6236.59869743,  6262.23149567,  6331.98393777,  6519.88013698,
        6571.18988623,  6593.81361997,  6556.44791832,  6539.61170035,
      

In [15]:
Metric_Comp(Y_vali, validation_data['close'].values)

0.98889694364628367

## Test prediction

In [16]:
test_data = pd.read_csv('..\\..\\Bitcoint_Data\\test_timeseries.csv')

In [17]:
Y_test = test_predict(model_log, test_data[['open','high','low']].values)

In [18]:
submission = pd.DataFrame(test_data['date'].values, columns = ['date'])
submission['close'] = Y_test

In [19]:
submission.to_csv('..\\..\\submission\\1st_time.csv', index=None)